In [1]:
##크롤링 대상 사이트 : https://www.computer.org (IEEE)
## https://ieeexplore.ieee.org 여기와 같은 자료

#사용할 라이브러리
import numpy as np
import pandas as pd
import requests # 크롤링
from bs4 import BeautifulSoup as bs # 크롤링
from multiprocessing import Pool # 멀티 프로세싱
from functools import partial # pool.map에서 여러 매개 변수를 받기 위함
import time # 시간 기록


In [3]:
#url 크롤링하는 함수 정의

#하나의 url이 주어지면 해당 태그의 url전부 추출하여 list로 반환
def get_link(url,tag_name):
    req = requests.get('https://www.computer.org'+url)
    soup = bs(req.text, 'html.parser')
    tag_list = soup.select(tag_name)       

    link = [] 
    for i in range(0,len(tag_list)):
        link.append(tag_list[i].get("href")) # url이 있는 href만 추출
        
    return link #list 반환

In [4]:

# computer science topic url list
topic_href = get_link('/csdl','a.transactionLink')


abstract_html = [] #abstract url을 저장할 리스트

pool = Pool(processes=32) # 프로세스 32개

#멀티 프로세싱을 이용하여 href를 추출하고, pool.map의 반환 값이 2차원 배열이기 때문에 1차원으로 합치는 sum함수 사용
#pool.map(function,list) : 해당 function에 list의 값들을 매핑

#topic url -> year url
#get_link의 tag_name 매개변수 값은 'a.volumeLink'
year_href = sum(pool.map(partial(get_link, tag_name='a.volumeLink'),topic_href),[]) # 해당 태그를 인자로 주고, 주어진 topic url들 속의 href들을 추출


In [5]:
len(year_href) # 모든 topic들의 년도 url개수

398

In [6]:
#year url -> months url
start = time.time()
months_href = sum(pool.map(partial(get_link, tag_name='a.issueLink'),year_href),[])
close = time.time()

print(str(close-start))

17.80996823310852


In [7]:
len(months_href) # 모든 months들의 url 개수

3211

In [8]:
#months url -> abstract url
start = time.time()
abstract_href = sum(pool.map(partial(get_link, tag_name='div.col-xs-12 > div > div > div > button > a'),months_href),[])
abstract_html.extend([ x for x in abstract_href if x[-8:] == "abs.html"]) # pdf와 javascript 제거
close = time.time()
print(str(close-start))

256.29295206069946


In [9]:
len(abstract_html) # 총 abstract url 개수

49653

In [10]:
# '\n'로 구분하여 전체 abstract url주소들을 파일로 저장
with open("./output/all_html.txt", 'w') as f: 
    for x in abstract_html:
        f.write("%s\n" % x)

In [11]:
#abstract url에서 topic,title,issn,abstract,date값 추출하는 함수
def croll_abs(link):
    req = requests.get('https://www.computer.org'+link)
    soup = bs(req.text, 'html.parser')
    journal = {}
    journal['topic'] = soup.select('a.breadCrumbLink')[1].get('title')
    journal['title'] = soup.select('div.abstractTitle > a > h2')[0].text
    journal['ISSN'] = (soup.select('div.abstractText')[1].text)[-9:]
    journal['abstract'] = soup.select('div.abstractText.abstractTextMB')[0].text
    journal['date'] = link[15:22]
    
    return journal

In [ ]:
pool = Pool(processes=64)
start = time.time()    
result = pool.map(croll_abs,abstract_html)
close = time.time()
print(str(close-start))